In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import csv
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
sample_submission=pd.read_csv('/kaggle/input/patient-healthcare/sample_submission.csv')
train=pd.read_csv('/kaggle/input/patient-healthcare/train.csv')
test=pd.read_csv('/kaggle/input/patient-healthcare/test.csv',index_col=False)
for name in train.select_dtypes(exclude='number').columns:
    le=LabelEncoder()
    label = le.fit_transform(train[name])
    train.drop(name, axis=1, inplace=True)
    train[name] = label
for name in test.select_dtypes(exclude='number').columns:
    le=LabelEncoder()
    label = le.fit_transform(test[name])
    test.drop(name, axis=1, inplace=True)
    test[name] = label
# train=train.dropna(axis=0,how='any')
# test=test.dropna(axis=0,how='any')
train=train.interpolate(method='linear',limit_direction='forward')
test=test.interpolate(method='linear',limit_direction='forward')
train_y=train['has_died']
train.pop('has_died')
ids=test['patient_id']
test=np.asarray(test).astype(np.float32)
train=np.asarray(train).astype(np.float32)
train_y=np.asarray(train_y).astype(np.float32)
train_X, test_X, train_Y, test_y = train_test_split(train, train_y, test_size=0.1, random_state=10)
clf=LogisticRegression(random_state=0,solver='lbfgs', max_iter=10000).fit(train_X, train_Y)
arr=clf.predict(test)
print(arr)
filename = "records.csv"
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(['patient_id','has_died'])
    i=0
    for id in ids:
        csvwriter.writerows([[id],[arr[i]]])
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[0. 0. 0. ... 0. 0. 0.]


In [2]:
len(arr)

27514

In [3]:
record=pd.read_csv('records.csv')
recording=record[record['patient_id'].eq(0.0)].index
record.drop(recording, inplace=True)
record.drop(labels='has_died', axis="columns", inplace=True)
record['has_died']=arr
record.to_csv('/kaggle/working/records.csv')

In [4]:
len(record)

27514